## Import dependencies

In [48]:
import numpy as np
import pandas as pd
import pprint

## Elliptic curve logic

### Logic for find order of concrete point

In [49]:
INFINITY = (0, 0)

In [50]:
def isPointValid(point):
    return point is not None and point is not np.nan

In [51]:
def arePointsEqual(p1, p2):
    return p1 == p2

In [52]:
def modulo_multiplicative_inverse(number, mod):
    # This will iterate from 0 to mod-1
    for i in range(0, mod):
        # If we have our multiplicative inverse then return it.
        if (number * i) % mod == 1:
            return i
    return -1

In [53]:
def calculate_s(numerator, denominator, mod):
    return (numerator * modulo_multiplicative_inverse(denominator, mod)) % mod

In [54]:
def sum(p1, p2, mod, a):
    if (not isPointValid(p1) or p1 == INFINITY): return p2
    if (not isPointValid(p2) or p2 == INFINITY): return p1
    s = 0
    if (arePointsEqual(p1, p2)):
        if (p1[1] == 0):
            return INFINITY
        numerator = 3*p1[0]*p1[0] + a
        denominator = 2*p1[1]
        s = calculate_s(numerator, denominator, mod)
    else:
        if (p1[0] == p2[0]):
            return INFINITY
        numerator = p2[1] - p1[1]
        denominator = p2[0] - p1[0]
        s = calculate_s(numerator, denominator, mod)    
        
    x = (s*s - p1[0] - p2[0]) % mod
    y = (s*(p1[0] - x) - p1[1]) % mod
    return (x, y)

In [55]:
def point_order(point, mod, a):
    points_of_sum = {}
    points_of_sum['P'] = point
    i = 1
    result = point
    while (result != INFINITY):
        result = sum(point, result, mod, a)
        i = i + 1
        key = str(i) + 'P'
        points_of_sum[key] = result
    return i, points_of_sum

### Logic for find all points

In [56]:
def get_modulas(mod):
    values = []
    for i in range(0, mod):
        values.append(i)
    return values

In [57]:
def get_y_square(a, b, mod):
    y_square = []
    for i in range(0, mod):
        result = (i**3 + a*i + b) % mod
        y_square.append(result)
    return y_square

In [58]:
def calculate_y_values(x, mod):
    y_value = None
    for i in range(0, mod // 2 + 1):
        r = i**2 % mod
        if (r == x):
            if (i == 0):
                y_value = str(i)
            else:
                y_value = "+-" + str(i)
    return y_value

In [59]:
def get_y_values(mod):
    y_values = []        
    for value in get_y_square(a, b, mod):
        y_values.append(calculate_y_values(value, mod))
    return y_values

In [60]:
def get_all_points_of_elliptic_curve(a, b, mod):
    df = pd.DataFrame(None, index=['x','y^2','y'], columns=np.arange(mod))
    df.loc['x'] = get_modulas(mod)
    df.loc['y^2'] = get_y_square(a, b, mod)
    df.loc['y'] = get_y_values(mod)
    return df

## Initialize params

### The elliptic curve is given in Weierstrass form
### y^2 = x^3 + a*x + b

### Get order of concrete point

In [65]:
point = (0, 1)
mod = 47
a = -5
oder_with_kist_of_points = point_order(point, mod, a)
bold_ANSI_start = '\033[1m'
bold_ANSI_end = '\033[0m'
print('Order of point ' + str(point) + ' is ' + bold_ANSI_start + str(oder_with_kist_of_points[0]) + bold_ANSI_end)
pprint.pprint(oder_with_kist_of_points[1], sort_dicts=False)

Order of point (0, 1) is 41
{'P': (0, 1),
 '2P': (18, 44),
 '3P': (7, 11),
 '4P': (43, 45),
 '5P': (31, 11),
 '6P': (23, 31),
 '7P': (5, 17),
 '8P': (9, 36),
 '9P': (45, 12),
 '10P': (44, 6),
 '11P': (11, 33),
 '12P': (13, 21),
 '13P': (8, 12),
 '14P': (24, 13),
 '15P': (35, 5),
 '16P': (33, 10),
 '17P': (30, 25),
 '18P': (12, 27),
 '19P': (41, 12),
 '20P': (42, 29),
 '21P': (42, 18),
 '22P': (41, 35),
 '23P': (12, 20),
 '24P': (30, 22),
 '25P': (33, 37),
 '26P': (35, 42),
 '27P': (24, 34),
 '28P': (8, 35),
 '29P': (13, 26),
 '30P': (11, 14),
 '31P': (44, 41),
 '32P': (45, 35),
 '33P': (9, 11),
 '34P': (5, 30),
 '35P': (23, 16),
 '36P': (31, 36),
 '37P': (43, 2),
 '38P': (7, 36),
 '39P': (18, 3),
 '40P': (0, 46),
 '41P': (0, 0)}


### Find sum of two points of an elliptic curve

In [62]:
point1 = (18, 44)
point2 = (11, 33)
mod = 47
a = -5
sum(point1, point2, mod, a)

(8, 12)

### Get all points of elliptic curve

In [63]:
a = -1
b = 0
mod = 7
df_points = get_all_points_of_elliptic_curve(a, b, mod)
df_points

,0,1,2,3,4,5,6
x,0,1,2,3,4,5,6
y^2,0,0,6,3,4,1,0
y,0,0,None,None,+-2,+-1,0


### Count of all points of elliptic curve + INIFINITY point

In [64]:
count = 1 # Add INFINITY point
y_values = df_points.loc['y']
for y_value in y_values:
    if (y_value is None):
        continue    
    if ('+-' in y_value):
        count = count + 2
    else:
        count = count + 1
count

8